In [1]:
import numpy as np
import pandas as pd

In [6]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# `actions_df`

In [9]:
actions_df = actions_df.reset_index('action')

In [44]:
def normalize_multiattack(x):
    df = x.set_index('action').copy()
    if 'Multiattack' in df.index:
        desc = df.loc['Multiattack'].desc
        if type(desc == pd.Series):
            print('hmm')
        actions = [action.lower() for action in df.index if action != 'Multiattack']
        print(x.name, desc.lower())
        print([action for action in actions if action in desc])

numerics = {'one': 1,
            'two': 2,
            'three': 3,
            'twice': 2}

In [45]:
actions_df.groupby('name').apply(normalize_multiattack)

hmm
Aboleth the aboleth makes three tentacle attacks.
['tentacle']
hmm
Aboleth the aboleth makes three tentacle attacks.
['tentacle']
hmm
Adult Black Dragon the dragon can use its frightful presence. it then makes three attacks: one with its bite and two with its claws.
['bite', 'claw']
hmm
Adult Blue Dracolich the dracolich can use its frightful presence. it then makes three attacks: one with its bite and two with its claws.
['bite', 'claw']
hmm
Adult Blue Dragon the dragon can use its frightful presence. it then makes three attacks: one with its bite and two with its claws.
['bite', 'claw']
hmm
Adult Brass Dragon the dragon can use its frightful presence. it then makes three attacks: one with its bite and two with its claws.
['bite', 'claw']
hmm
Adult Bronze Dragon the dragon can use its frightful presence. it then makes three attacks: one with its bite and two with its claws.
['bite', 'claw']
hmm
Adult Copper Dragon the dragon can use its frightful presence. it then makes three atta

TypeError: _get_axis() missing 1 required positional argument: 'axis'

In [ ]:
outliers = ['Animated Armor the armor makes two melee attacks.'
           ]